# Quaternion: Rotation of Cube

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
# quaternion multiplication function
def QbyQ(q1, q2): # q = [w, x, y, z]
    w = q1[0]*q2[0] - q1[1]*q2[1] - q1[2]*q2[2] - q1[3]*q2[3]
    i = q1[0]*q2[1] + q1[1]*q2[0] + q1[2]*q2[3] - q1[3]*q2[2]
    j = q1[0]*q2[2] - q1[1]*q2[3] + q1[2]*q2[0] + q1[3]*q2[1]
    k = q1[0]*q2[3] + q1[1]*q2[2] - q1[2]*q2[1] + q1[3]*q2[0]
    return np.array([w, i, j, k])

# quaternion rotation function
def QPQc(P, UV, t): # P: quaternion, UV: unit vector, t: rotation angle theta
    q =  [np.cos(t/2),  UV[0] * np.sin(t/2),  UV[1] * np.sin(t/2),  UV[2] * np.sin(t/2)]
    qc = [np.cos(t/2), -UV[0] * np.sin(t/2), -UV[1] * np.sin(t/2), -UV[2] * np.sin(t/2)]
    qP = QbyQ(q, P)
    result = QbyQ(qP, qc)
    return result

# plot unit sphere
def plot_base(elev=25, azim=-70):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=elev, azim=azim)
    ax.set(xlim=(-1, 1), ylim=(-1 ,1), zlim=(-1, 1))
    ax.set(xlabel='X', ylabel='Y', zlabel='Z')
    ax.xaxis.pane.fill = ax.yaxis.pane.fill = ax.zaxis.pane.fill = False

    t = np.linspace(0, 2*np.pi, 128+1)
    alpha = 0.7
    ax.plot(np.cos(t), np.sin(t), [0]*len(t), linestyle=':', c='red', alpha=alpha)
    ax.plot(np.cos(t), [0]*len(t), np.sin(t), linestyle=':', c='red', alpha=alpha)
    ax.plot([0]*len(t), np.cos(t), np.sin(t), linestyle=':', c='red', alpha=alpha)
    ax.plot([-1, 1], [0, 0], [0, 0], linestyle=':', c='red', alpha=alpha)
    ax.plot([0, 0], [-1, 1], [0, 0], linestyle=':', c='red', alpha=alpha)
    ax.plot([0, 0], [0, 0], [-1, 1], linestyle=':', c='red', alpha=alpha)
    return ax

## Rotating Cube with ipywidgets

In [3]:
from ipywidgets import interact
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

@interact(theta=(-np.pi, np.pi, np.pi/100), w=(0.1, 1, 0.1), l=(0.1, 1, 0.1), h=(0.1, 1, 0.1)) # theta: rotation angle on unit vector
def cube(theta=0, w=0.5, l=0.5, h=0.5):
    scale = np.array([w, l, h])  # scale of the cube
    pos = np.array([-w/2, -l/2, 1])  # position of the cube
    UV = [1, 0, 0]                     # unit vector: rotation axis

    CUBE = np.array([[0,0,0],[1,0,0],[1,1,0],[0,1,0], 
                     [0,0,1],[1,0,1],[1,1,1],[0,1,1]]) * scale + pos            # cube vertices with scale and position
    CUBEs = np.vstack([np.linalg.norm(c) for c in CUBE])                        # scalar
    CUBEn = np.array([v / s if s != 0 else v * 0 for v, s in zip(CUBE, CUBEs)]) # normalization
    QCUBE = np.insert(CUBEn, 0, 0, axis=1)                                      # quaternion

    UV = np.array(UV) / np.linalg.norm(UV)                    # normalization of unit vector

    QCUBE_R = np.array([QPQc(qc, UV, theta) for qc in QCUBE]) # quaternion rotation
    CX, CY, CZ = (QCUBE_R[:, 1:] * CUBEs).T                   # vectors of cube

    ax = plot_base()
    ax.plot([0, UV[0]], [0, UV[1]], [0, UV[2]], c='r')
    ax.scatter(UV[0], UV[1], UV[2], c='r')
    ax.plot(np.r_[CX[:4], CX[0]], np.r_[CY[:4], CY[0]], np.r_[CZ[:4], CZ[0]], c='b')         # 4 bottom edges
    ax.plot(np.r_[CX[4:], CX[4]], np.r_[CY[4:], CY[4]], np.r_[CZ[4:], CZ[4]], c='g')         # 4 top edges
    [ax.plot([CX[i], CX[i+4]], [CY[i], CY[i+4]], [CZ[i], CZ[i+4]], c='m') for i in range(4)] # 4 colomns edges
    
    bottom = list(zip(CX[:4], CY[:4], CZ[:4]))
    ax.add_collection3d(Poly3DCollection([bottom], color='#ddddff'))  # fill in color : bottom side
    top = list(zip(CX[4:], CY[4:], CZ[4:]))
    ax.add_collection3d(Poly3DCollection([top], color='#ddffdd'))     # fill in color : top side

interactive(children=(FloatSlider(value=0.0, description='theta', max=3.141592653589793, min=-3.14159265358979…

In [3]:
from ipywidgets import interact
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

@interact(theta=(-np.pi, np.pi, np.pi/100), ux=(-1, 1, 0.1), uy=(-1, 1, 0.1), uz=(-1, 1, 0.1)) # theta: rotation angle on unit vector
def cube(theta=0, ux=1, uy=0, uz=0):
    scale = np.array([1/2, 1/2, 1/2])  # scale of the cube
    pos = np.array([-1/4, -1/4, 1])  # position of the cube
    UV = [ux, uy, uz]                     # unit vector: rotation axis

    CUBE = np.array([[0,0,0],[1,0,0],[1,1,0],[0,1,0], 
                     [0,0,1],[1,0,1],[1,1,1],[0,1,1]]) * scale + pos            # cube vertices with scale and position
    CUBEs = np.vstack([np.linalg.norm(c) for c in CUBE])                        # scalar
    CUBEn = np.array([v / s if s != 0 else v * 0 for v, s in zip(CUBE, CUBEs)]) # normalization
    QCUBE = np.insert(CUBEn, 0, 0, axis=1)                                      # quaternion

    UV = np.array(UV) / np.linalg.norm(UV)                    # normalization of unit vector

    QCUBE_R = np.array([QPQc(qc, UV, theta) for qc in QCUBE]) # quaternion rotation
    CX, CY, CZ = (QCUBE_R[:, 1:] * CUBEs).T                   # vectors of cube

    ax = plot_base()
    ax.plot([0, UV[0]], [0, UV[1]], [0, UV[2]], c='r')
    ax.scatter(UV[0], UV[1], UV[2], c='r')
    ax.plot(np.r_[CX[:4], CX[0]], np.r_[CY[:4], CY[0]], np.r_[CZ[:4], CZ[0]], c='b')         # 4 bottom edges
    ax.plot(np.r_[CX[4:], CX[4]], np.r_[CY[4:], CY[4]], np.r_[CZ[4:], CZ[4]], c='g')         # 4 top edges
    [ax.plot([CX[i], CX[i+4]], [CY[i], CY[i+4]], [CZ[i], CZ[i+4]], c='m') for i in range(4)] # 4 colomns edges
    ax.text(ux, uy, uz, s='  unit vector')
    
    bottom = list(zip(CX[:4], CY[:4], CZ[:4]))
    ax.add_collection3d(Poly3DCollection([bottom], color='#ddddff'))  # fill in color : bottom side
    top = list(zip(CX[4:], CY[4:], CZ[4:]))
    ax.add_collection3d(Poly3DCollection([top], color='#ddffdd'))     # fill in color : top side

interactive(children=(FloatSlider(value=0.0, description='theta', max=3.141592653589793, min=-3.14159265358979…